In [33]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

In [34]:
print("Load the data.")
df_x_train = pd.read_csv('X_train.csv',header = 0 ,index_col=0)
df_y_train = pd.read_csv('y_train.csv',header = 0 ,index_col=0)
df_x_test = pd.read_csv('X_test.csv',header = 0 ,index_col=0)
x_train = df_x_train.values
y_train = df_y_train['y'].values
x_test = df_x_test.values

Load the data.


In [35]:
print('Check training data.')
print(df_x_train.head())
print(df_x_train.shape)
print('There are %d NAN values.'%np.sum(np.isnan(x_train)))
print(df_y_train.groupby('y').size())

Check training data.
           x0        x1        x2        x3        x4        x5        x6  \
id                                                                          
0.0 -1.099144  0.918165 -1.227394  0.887061  1.182734 -0.371802 -0.127331   
1.0  0.366477  0.933802  0.061234  0.430073  1.029673 -0.613771  0.364698   
2.0 -0.425879 -0.802152  0.002718  0.820036  1.490237 -0.888121  0.769524   
3.0  0.822922 -0.843041 -0.734624 -0.000040  0.123783 -1.777226  0.364601   
4.0 -0.285342 -0.641116  0.436524 -0.618663  0.319982 -1.160489  0.528379   

           x7        x8        x9  ...      x990      x991      x992  \
id                                 ...                                 
0.0  1.201702  1.825839  2.944655  ... -1.111422  1.566261 -1.656656   
1.0  1.140867  0.182811 -0.344876  ... -0.672386 -0.003721 -0.496326   
2.0  1.059020  0.854806 -0.077359  ... -1.031358 -0.085724 -0.557461   
3.0  0.425521  1.265122  0.734897  ... -0.118974  1.271367 -1.518161   
4.0  0.

In [36]:
# standardization
print('Standardize the data.')
scaler = preprocessing.StandardScaler().fit(x_train)
x_train =scaler.transform(x_train)
x_test=scaler.transform(x_test)

Standardize the data.


In [37]:
# print("Split data into training and validation sets.")
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2,stratify=y_train)

In [38]:
# subsampling
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import RandomUnderSampler

model_smote = SMOTE()
x_over_train, y_over_train = model_smote.fit_sample(x_train,y_train)

model_RandomUnderSampler = RandomUnderSampler() 
x_down_train, y_down_train =model_RandomUnderSampler.fit_sample(x_train,y_train)
print('After over-sampling:\n',pd.DataFrame(y_over_train,columns=['y']).groupby('y').size())
print('After down-sampling:\n',pd.DataFrame(y_down_train,columns=['y']).groupby('y').size())


After over-sampling:
 y
0    3600
1    3600
2    3600
dtype: int64
After down-sampling:
 y
0    600
1    600
2    600
dtype: int64


In [39]:
#feature selection
from sklearn.feature_selection import SelectFromModel, SelectKBest,f_classif,mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor

def feature_selection(x_train, y_train, x_test, method="rf", max_features=200):
    if(method == 'rf'):
        sel = SelectFromModel(RandomForestClassifier(n_estimators=100,class_weight='balanced'), threshold=-np.inf, max_features=max_features)
        sel.fit(x_train, y_train)
        indicator = sel.get_support()
        index = np.where(indicator==False)[0]

        print("select %d features!" % (indicator.shape[0] - index.shape[0]))
        x_train_fs = np.delete(x_train, index, axis=1)
        x_test_fs = np.delete(x_test, index, axis=1)
        return x_train_fs, x_test_fs
    
    elif(method == 'xgb'):
        sel = SelectFromModel(XGBRegressor(n_estimators=100, learning_rate=0.1, gamma=0.01, subsample=0.8, colsample_bytree=1, min_child_weight=5, max_depth=3), threshold=-np.inf, max_features=max_features)
        sel.fit(x_train, y_train)
        indicator = sel.get_support()
        index = np.where(indicator==False)[0]

        print("select %d features!" % (indicator.shape[0] - index.shape[0]))
        x_train_fs = np.delete(x_train, index, axis=1)
        x_test_fs = np.delete(x_test, index, axis=1)
        return x_train_fs, x_test_fs

    elif(method == 'freg'):
        #x_train = SelectKBest(f_regression, k=200).fit_transform(x_train, y_train)
        sel = SelectKBest(f_classif, k=max_features)
        sel.fit(x_train, y_train)
        indicator = sel.get_support()
        index = np.where(indicator==False)[0]

        print("select %d features!" % (indicator.shape[0] - index.shape[0]))
        x_train_fs = np.delete(x_train, index, axis=1)
        x_test_fs = np.delete(x_test, index, axis=1)
        return x_train_fs, x_test_fs
    
    elif(method == 'mir'):
        sel = SelectKBest(mutual_info_classif, k=max_features)
        sel.fit(x_train, y_train)
        indicator = sel.get_support()
        index = np.where(indicator==False)[0]

        print("select %d features!" % (indicator.shape[0] - index.shape[0]))
        x_train_fs = np.delete(x_train, index, axis=1)
        x_test_fs = np.delete(x_test, index, axis=1)
        return x_train_fs, x_test_fs
    
fs_x_down_train,fs_x_down_test = feature_selection(x_down_train,y_down_train,x_test,"rf",500)
fs_x_over_train,fs_x_over_test = feature_selection(x_over_train,y_over_train,x_test,"rf",500)
fs_x_train,fs_x_test = feature_selection(x_train,y_train,x_test,"rf",500) 


select 500 features!
select 500 features!
select 500 features!


In [72]:
# remove outliers
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
def outlier_detection(x_train, y_train, method='isoforest'):
    if(method == 'isoforest'):
#         rng = np.random.RandomState(42)
        clf = IsolationForest(behaviour='new', max_samples=1000,
#                           random_state=rng, 
                              contamination='auto')
        clf.fit(x_train)
        indicator = clf.predict(x_train)
        index = np.where(indicator == -1)[0]
        print("detect %d outliers in training set!" % (index.shape[0]))

        x_train_clean = np.delete(x_train, index, axis=0)
        y_train_clean = np.delete(y_train, index, axis=0)

        return x_train_clean, y_train_clean
    
    elif(method == 'lof'):
        clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
        clf.fit(x_train)
        indicator = clf.fit_predict(x_train)
        index = np.where(indicator == -1)[0]
        print("detect %d outliers in training set!" % (index.shape[0]))

        x_train_clean = np.delete(x_train, index, axis=0)
        y_train_clean = np.delete(y_train, index, axis=0)
        return x_train_clean,y_train_clean
    

x_down_train,y_down_train = outlier_detection(x_down_train,y_down_train,method='isoforest')

detect 65 outliers in training set!


In [41]:
# using customized evaluation metric(for early stopping)
# TODO 
#  using class weight for each data point to manually balance the dataset(xgboost use one v all strategy for multiclass problem)
# why weight samples would cause imprecise evaluation of the class probability ???

def custom_error(preds, train_data):
    labels = train_data.get_label()
    if preds.ndim == 2:
        pred_label = [np.argmax(x) for x in preds]
    elif preds.ndim == 1:
        pred_label = [1 if x>=0.5 else 0 for x in preds]
    else:
        print("Unknown preds structure.")
        return
    return '1-BMAC', 1-balanced_accuracy_score(labels, pred_label)

import xgboost as xgb
def train_xgb(x_train,y_train,x_test,params,n_iter=10,choose_best=True,obj=None,feval=custom_error):
    best = 0
    results = []
    for i in range(n_iter):
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train,test_size=0.2)
        xgb_train = xgb.DMatrix(x_train, label=y_train)
        xgb_val = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(xgb_train, 'train'), (xgb_val, 'valid')]
        bst = xgb.train(params, 
                        xgb_train, 
                        50,  # number of iterations
                        watchlist, 
                        early_stopping_rounds=15,
                        obj=obj,
                        feval=feval)
        y_val_p = bst.predict(xgb_val)
        result = balanced_accuracy_score(y_val, [np.argmax(x) for x in y_val_p])
        results.append(result)
        print("val_BMAC:",result)
        
        if choose_best & (result <= best):
                continue
        best = result
        xgb_test = xgb.DMatrix(x_test)
        y_pred = bst.predict(xgb_test)
        f = open("submission_xgb{0:.4f}.csv".format(best), "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,np.argmax(x)))
        f.close()
    return results

# specify your configurations as a dict
params = {'booster' : 'gbtree',  # 'dart'
          'num_class':3,
          
          'min_child_weight': 5, 
          'max_depth': 6,
          
          'learning_rate':0.05,
          'colsample_bytree': 0.5, 
          'subsample': 0.5, 
          'alpha': 1., 

          # 'eval_metric': ['merror','mlogloss'], 
          'objective': 'multi:softprob'}

In [73]:
# with down sampling data
params = {'booster' : 'gbtree',  # 'dart'
          'num_class':3,
          
          'min_child_weight': 5, 
          'max_depth': 6,
          
          'learning_rate':0.05,
          'colsample_bytree': 0.5, 
          'subsample': 0.5, 
          'alpha': 1., 

          # 'eval_metric': ['merror','mlogloss'], 
          'objective': 'multi:softprob'}
train_xgb(x_down_train,y_down_train,x_test,params,n_iter=10,choose_best=True)

[0]	train-merror:0.299732	valid-merror:0.480427	train-1-BMAC:0.300894	valid-1-BMAC:0.479409
Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.235504	valid-merror:0.455516	train-1-BMAC:0.234818	valid-1-BMAC:0.450337
[2]	train-merror:0.204282	valid-merror:0.412811	train-1-BMAC:0.203935	valid-1-BMAC:0.408767
[3]	train-merror:0.182872	valid-merror:0.423488	train-1-BMAC:0.182331	valid-1-BMAC:0.416408
[4]	train-merror:0.174844	valid-merror:0.423488	train-1-BMAC:0.174939	valid-1-BMAC:0.416829
[5]	train-merror:0.158787	valid-merror:0.405694	train-1-BMAC:0.159573	valid-1-BMAC:0.401709
[6]	train-merror:0.152542	valid-merror:0.419929	train-1-BMAC:0.154627	valid-1-BMAC:0.417735
[7]	train-merror:0.146298	valid-merror:0.41637	train-1-BMAC:0.148724	valid-1-BMAC:0.413785
[8]	train-merror:0.140054	valid-merror:0.441281	train-1-BMAC:0.142748	valid-1-BMAC:0.436707
[9]	train-merror:0.12934

[27]	train-merror:0.072626	valid-merror:0.338889	train-1-BMAC:0.075196	valid-1-BMAC:0.342999
[28]	train-merror:0.074022	valid-merror:0.344444	train-1-BMAC:0.077228	valid-1-BMAC:0.349666
[29]	train-merror:0.074022	valid-merror:0.344444	train-1-BMAC:0.076871	valid-1-BMAC:0.347974
[30]	train-merror:0.071229	valid-merror:0.338889	train-1-BMAC:0.073878	valid-1-BMAC:0.341308
[31]	train-merror:0.069832	valid-merror:0.344444	train-1-BMAC:0.073028	valid-1-BMAC:0.347974
[32]	train-merror:0.067039	valid-merror:0.333333	train-1-BMAC:0.068963	valid-1-BMAC:0.336017
[33]	train-merror:0.065642	valid-merror:0.35	train-1-BMAC:0.06759	valid-1-BMAC:0.351574
[34]	train-merror:0.064246	valid-merror:0.35	train-1-BMAC:0.06663	valid-1-BMAC:0.351258
[35]	train-merror:0.058659	valid-merror:0.338889	train-1-BMAC:0.060755	valid-1-BMAC:0.339616
[36]	train-merror:0.054469	valid-merror:0.344444	train-1-BMAC:0.056912	valid-1-BMAC:0.344907
[37]	train-merror:0.053073	valid-merror:0.35	train-1-BMAC:0.055237	valid-1-BMAC:

[13]	train-merror:0.107221	valid-merror:0.417391	train-1-BMAC:0.109206	valid-1-BMAC:0.412844
[14]	train-merror:0.107221	valid-merror:0.408696	train-1-BMAC:0.107999	valid-1-BMAC:0.404714
[15]	train-merror:0.098468	valid-merror:0.408696	train-1-BMAC:0.100012	valid-1-BMAC:0.402233
[16]	train-merror:0.102845	valid-merror:0.4	train-1-BMAC:0.104413	valid-1-BMAC:0.394297
[17]	train-merror:0.105033	valid-merror:0.408696	train-1-BMAC:0.106679	valid-1-BMAC:0.402233
[18]	train-merror:0.100656	valid-merror:0.4	train-1-BMAC:0.101984	valid-1-BMAC:0.396971
[19]	train-merror:0.09628	valid-merror:0.434783	train-1-BMAC:0.097191	valid-1-BMAC:0.43139
[20]	train-merror:0.09628	valid-merror:0.4	train-1-BMAC:0.097289	valid-1-BMAC:0.39449
[21]	train-merror:0.098468	valid-merror:0.417391	train-1-BMAC:0.099816	valid-1-BMAC:0.410557
[22]	train-merror:0.102845	valid-merror:0.391304	train-1-BMAC:0.104511	valid-1-BMAC:0.38388
[23]	train-merror:0.094092	valid-merror:0.408696	train-1-BMAC:0.095219	valid-1-BMAC:0.3999

[1]	train-merror:0.284247	valid-merror:0.575342	train-1-BMAC:0.282962	valid-1-BMAC:0.562156
[2]	train-merror:0.229452	valid-merror:0.493151	train-1-BMAC:0.230063	valid-1-BMAC:0.47944
[3]	train-merror:0.19863	valid-merror:0.479452	train-1-BMAC:0.199628	valid-1-BMAC:0.470465
[4]	train-merror:0.184932	valid-merror:0.452055	train-1-BMAC:0.185804	valid-1-BMAC:0.441928
[5]	train-merror:0.14726	valid-merror:0.452055	train-1-BMAC:0.147888	valid-1-BMAC:0.438557
[6]	train-merror:0.184932	valid-merror:0.438356	train-1-BMAC:0.184196	valid-1-BMAC:0.429582
[7]	train-merror:0.157534	valid-merror:0.438356	train-1-BMAC:0.157148	valid-1-BMAC:0.425261
[8]	train-merror:0.136986	valid-merror:0.452055	train-1-BMAC:0.135393	valid-1-BMAC:0.441928
[9]	train-merror:0.143836	valid-merror:0.424658	train-1-BMAC:0.143623	valid-1-BMAC:0.412441
[10]	train-merror:0.133562	valid-merror:0.438356	train-1-BMAC:0.133036	valid-1-BMAC:0.425261
[11]	train-merror:0.116438	valid-merror:0.452055	train-1-BMAC:0.114367	valid-1-BMA

[33]	train-merror:0.053763	valid-merror:0.319149	train-1-BMAC:0.053344	valid-1-BMAC:0.330317
Stopping. Best iteration:
[18]	train-merror:0.075269	valid-merror:0.297872	train-1-BMAC:0.077406	valid-1-BMAC:0.310709

val_BMAC: 0.669683257918552
[0]	train-merror:0.378378	valid-merror:0.5	train-1-BMAC:0.36985	valid-1-BMAC:0.497669
Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.283784	valid-merror:0.5	train-1-BMAC:0.279786	valid-1-BMAC:0.491175
[2]	train-merror:0.175676	valid-merror:0.447368	train-1-BMAC:0.169001	valid-1-BMAC:0.443556
[3]	train-merror:0.155405	valid-merror:0.394737	train-1-BMAC:0.152189	valid-1-BMAC:0.387612
[4]	train-merror:0.135135	valid-merror:0.447368	train-1-BMAC:0.132608	valid-1-BMAC:0.443556
[5]	train-merror:0.108108	valid-merror:0.421053	train-1-BMAC:0.105947	valid-1-BMAC:0.419747
[6]	train-merror:0.128378	valid-merror:0.394737	train-1-BMAC:0.128967

[0.5594405594405595,
 0.6554054929394999,
 0.647366342888731,
 0.600524109014675,
 0.6217697444831591,
 0.6456447963800905,
 0.5324311490978157,
 0.6275252525252525,
 0.669683257918552,
 0.5308025308025308]

In [43]:
# TODO weighted samples
# with original data
train_xgb(x_train,y_train,x_test,params,n_iter=10,choose_best=True)

[0]	train-merror:0.17474	valid-merror:0.244792	train-1-BMAC:0.378588	valid-1-BMAC:0.497685
Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.152865	valid-merror:0.215625	train-1-BMAC:0.357292	valid-1-BMAC:0.484722
[2]	train-merror:0.140885	valid-merror:0.219792	train-1-BMAC:0.334606	valid-1-BMAC:0.505093
[3]	train-merror:0.139323	valid-merror:0.216667	train-1-BMAC:0.332755	valid-1-BMAC:0.496759
[4]	train-merror:0.134115	valid-merror:0.211458	train-1-BMAC:0.322917	valid-1-BMAC:0.485185
[5]	train-merror:0.129948	valid-merror:0.211458	train-1-BMAC:0.319329	valid-1-BMAC:0.494444
[6]	train-merror:0.125781	valid-merror:0.208333	train-1-BMAC:0.310532	valid-1-BMAC:0.490741
[7]	train-merror:0.128906	valid-merror:0.208333	train-1-BMAC:0.319444	valid-1-BMAC:0.493056
[8]	train-merror:0.127604	valid-merror:0.207292	train-1-BMAC:0.318866	valid-1-BMAC:0.492593
[9]	train-merror:0.12578

[29]	train-merror:0.087098	valid-merror:0.185366	train-1-BMAC:0.221497	valid-1-BMAC:0.432206
[30]	train-merror:0.087098	valid-merror:0.188618	train-1-BMAC:0.221497	valid-1-BMAC:0.440864
[31]	train-merror:0.085063	valid-merror:0.188618	train-1-BMAC:0.216973	valid-1-BMAC:0.440864
[32]	train-merror:0.083842	valid-merror:0.185366	train-1-BMAC:0.214621	valid-1-BMAC:0.432206
[33]	train-merror:0.083028	valid-merror:0.190244	train-1-BMAC:0.213354	valid-1-BMAC:0.437981
[34]	train-merror:0.0814	valid-merror:0.186992	train-1-BMAC:0.209916	valid-1-BMAC:0.432929
[35]	train-merror:0.079772	valid-merror:0.190244	train-1-BMAC:0.204668	valid-1-BMAC:0.437981
[36]	train-merror:0.078144	valid-merror:0.18374	train-1-BMAC:0.200325	valid-1-BMAC:0.427877
[37]	train-merror:0.074888	valid-merror:0.180488	train-1-BMAC:0.192544	valid-1-BMAC:0.422824
[38]	train-merror:0.070818	valid-merror:0.185366	train-1-BMAC:0.181686	valid-1-BMAC:0.4286
[39]	train-merror:0.070004	valid-merror:0.186992	train-1-BMAC:0.179514	vali

[14]	train-merror:0.099237	valid-merror:0.206107	train-1-BMAC:0.253134	valid-1-BMAC:0.516984
[15]	train-merror:0.099873	valid-merror:0.208651	train-1-BMAC:0.253425	valid-1-BMAC:0.523786
[16]	train-merror:0.099237	valid-merror:0.206107	train-1-BMAC:0.253142	valid-1-BMAC:0.516984
[17]	train-merror:0.097964	valid-merror:0.211196	train-1-BMAC:0.24975	valid-1-BMAC:0.530589
[18]	train-merror:0.096056	valid-merror:0.211196	train-1-BMAC:0.243238	valid-1-BMAC:0.530589
Stopping. Best iteration:
[3]	train-merror:0.125318	valid-merror:0.206107	train-1-BMAC:0.302924	valid-1-BMAC:0.48862

val_BMAC: 0.469410815173527
[0]	train-merror:0.163882	valid-merror:0.222222	train-1-BMAC:0.354193	valid-1-BMAC:0.463672
Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.143994	valid-merror:0.228571	train-1-BMAC:0.336508	valid-1-BMAC:0.494394
[2]	train-merror:0.123309	valid-merror:0.212698	train-1-B

[5]	train-merror:0.108865	valid-merror:0.21118	train-1-BMAC:0.279904	valid-1-BMAC:0.524931
[6]	train-merror:0.108865	valid-merror:0.21118	train-1-BMAC:0.279955	valid-1-BMAC:0.511019
[7]	train-merror:0.107309	valid-merror:0.21118	train-1-BMAC:0.282687	valid-1-BMAC:0.524931
[8]	train-merror:0.107309	valid-merror:0.204969	train-1-BMAC:0.275737	valid-1-BMAC:0.508264
[9]	train-merror:0.11353	valid-merror:0.204969	train-1-BMAC:0.292404	valid-1-BMAC:0.508264
[10]	train-merror:0.104199	valid-merror:0.204969	train-1-BMAC:0.27093	valid-1-BMAC:0.522176
[11]	train-merror:0.104199	valid-merror:0.204969	train-1-BMAC:0.27093	valid-1-BMAC:0.522176
[12]	train-merror:0.096423	valid-merror:0.204969	train-1-BMAC:0.2502	valid-1-BMAC:0.522176
[13]	train-merror:0.101089	valid-merror:0.204969	train-1-BMAC:0.262649	valid-1-BMAC:0.508264
[14]	train-merror:0.097978	valid-merror:0.198758	train-1-BMAC:0.254418	valid-1-BMAC:0.491598
[15]	train-merror:0.096423	valid-merror:0.192547	train-1-BMAC:0.250149	valid-1-BMAC

[0.513425925925926,
 0.5503472222222223,
 0.588716417349823,
 0.5265523420261525,
 0.469410815173527,
 0.502578589019267,
 0.5284800449450987,
 0.5756291390728476,
 0.5639118457300275,
 0.5]

In [44]:
# using customized loss/objective function to learn more on difficult samples
# how to implement it for multiclass problem directly ??

from scipy.misc import derivative
def focal_loss_lgb(y_pred,dtrain,alpha,gamma):
    a,g = alpha,gamma
    y_true = dtrain.get_label()
    def fl(y_pred):
        p = np.array([x if y_true[i]==1 else (1-x) for i,x in enumerate(y_pred)])
        return -a*((1-p)**g)*np.log(p)
    grad = derivative(fl, y_pred, n=1, dx=1e-6)
    hess = derivative(fl, y_pred, n=2, dx=1e-6)
    return grad, hess

def focal_loss_lgb_eval_error(y_pred, dtrain, alpha, gamma):
  a,g = alpha, gamma
  y_true = dtrain.get_label()
  p = 1/(1+np.exp(-y_pred))
  loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
  # (eval_name, eval_result, is_higher_better)
  return 'focal_loss', np.mean(loss)

focal_loss = lambda x,y: focal_loss_lgb(x, y, alpha=0.25, gamma=5.)
focal_loss_eval = lambda x,y: focal_loss_lgb_eval_error(x, y, alpha=0.25, gamma=1.)

In [45]:
def train_onevsall(x_train,y_train,x_test,learner,params1,params2=None,feval_val=balanced_accuracy_score):  # params : must include feval, how to force this? ?
    labels = np.unique(y_train)
    y_final_pred = np.zeros((x_test.shape[0],labels.shape[0]))
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train,test_size=0.1)
    y_val_final_p = np.zeros((x_val.shape[0],labels.shape[0]))
    for i,l in enumerate(labels):
        print('*************************************************')
        print('Learner No. %d:\n' %(i+1))
        y_train_masked = 1*(y_train==l)
        y_pred,y_val_p = learner(x_train,y_train_masked,x_val,x_test,params1,params2,feval_val=feval_val)
        # evaluation
        y_val_final_p[:,i] = y_val_p
        y_final_pred[:,i] = y_pred
    label_pred = [labels[np.argmax(x)] for x in y_final_pred]
    label_val_p = [labels[np.argmax(x)] for x in y_val_final_p]
    result = feval_val(y_val, label_val_p)
    print('----------ove vs all classifier------------')
    print('%s on validation data:%f' %(feval_val.__name__,result))
    return label_pred, result

def xgbt(x_train,y_train,x_val,x_test,params1=None,params2=None,feval_val=balanced_accuracy_score):
    x_train, x_val_inner, y_train, y_val_inner = train_test_split(x_train, y_train, stratify=y_train,test_size=0.1)
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dval = xgb.DMatrix(x_val_inner, label=y_val_inner)
    dtest = xgb.DMatrix(x_test)
    dval_outer = xgb.DMatrix(x_val)
    watchlist = [(dval, 'eval'), (dtrain, 'train')]
    bst = xgb.train(params1, dtrain, 80 ,watchlist,**params2)   
    y_val_p = bst.predict(dval)
    result = feval_val(y_val_inner, [1 if x>=0.5 else 0 for x in y_val_p])
    print('----------binary classifier------------')
    print('%s on validation data:%f' %(feval_val.__name__,result))
    return bst.predict(dtest),bst.predict(dval_outer)

In [46]:
# with logistic loss
params1 = {'booster' : 'gbtree',  # 'dart'
          'max_depth': 10,
          'objective':'binary:logistic',
          'learning_rate':0.05}
params2 = {'feval':custom_error}     #'focal_loss_eval'         
y_pred, val_result = train_onevsall(x_down_train,y_down_train,x_test,xgbt,params1=params1,params2=params2)
 
print('Write to submission file.')
f = open("submission_xgb{0:.4f}.csv".format(val_result), "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()

*************************************************
Learner No. 1:

[0]	eval-error:0.356688	train-error:0.069751	eval-1-BMAC:0.367404	train-1-BMAC:0.088634
[1]	eval-error:0.350318	train-error:0.032028	eval-1-BMAC:0.394732	train-1-BMAC:0.041655
[2]	eval-error:0.375796	train-error:0.019217	eval-1-BMAC:0.41415	train-1-BMAC:0.024993
[3]	eval-error:0.356688	train-error:0.024911	eval-1-BMAC:0.395182	train-1-BMAC:0.03426
[4]	eval-error:0.369427	train-error:0.019217	eval-1-BMAC:0.40489	train-1-BMAC:0.026472
[5]	eval-error:0.363057	train-error:0.015658	eval-1-BMAC:0.400036	train-1-BMAC:0.022281
[6]	eval-error:0.369427	train-error:0.01637	eval-1-BMAC:0.400485	train-1-BMAC:0.022331
[7]	eval-error:0.33758	train-error:0.0121	eval-1-BMAC:0.376214	train-1-BMAC:0.017598
[8]	eval-error:0.33121	train-error:0.010676	eval-1-BMAC:0.362549	train-1-BMAC:0.015528
[9]	eval-error:0.33758	train-error:0.009964	eval-1-BMAC:0.380618	train-1-BMAC:0.014493
[10]	eval-error:0.324841	train-error:0.009253	eval-1-BMAC:0.362

[19]	eval-error:0.197452	train-error:0.00427	eval-1-BMAC:0.243859	train-1-BMAC:0.006818
[20]	eval-error:0.184713	train-error:0.003559	eval-1-BMAC:0.229025	train-1-BMAC:0.005682
[21]	eval-error:0.191083	train-error:0.003559	eval-1-BMAC:0.233655	train-1-BMAC:0.005682
[22]	eval-error:0.171975	train-error:0.002847	eval-1-BMAC:0.214191	train-1-BMAC:0.004545
[23]	eval-error:0.178344	train-error:0.002847	eval-1-BMAC:0.224395	train-1-BMAC:0.004545
[24]	eval-error:0.171975	train-error:0.002847	eval-1-BMAC:0.214191	train-1-BMAC:0.004545
[25]	eval-error:0.171975	train-error:0.002847	eval-1-BMAC:0.214191	train-1-BMAC:0.004545
[26]	eval-error:0.159236	train-error:0.002847	eval-1-BMAC:0.199358	train-1-BMAC:0.004545
[27]	eval-error:0.165605	train-error:0.002135	eval-1-BMAC:0.209562	train-1-BMAC:0.003409
[28]	eval-error:0.165605	train-error:0.002847	eval-1-BMAC:0.209562	train-1-BMAC:0.004545
[29]	eval-error:0.171975	train-error:0.002135	eval-1-BMAC:0.219766	train-1-BMAC:0.003409
[30]	eval-error:0.1656

[37]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[38]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[39]	eval-error:0.210191	train-error:0	eval-1-BMAC:0.243887	train-1-BMAC:0
[40]	eval-error:0.210191	train-error:0	eval-1-BMAC:0.243887	train-1-BMAC:0
[41]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[42]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[43]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[44]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[45]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[46]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[47]	eval-error:0.203822	train-error:0	eval-1-BMAC:0.234628	train-1-BMAC:0
[48]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.225369	train-1-BMAC:0
[49]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.225369	train-1-BMAC:0
[50]	eval-error:0.197452	

In [47]:
# with focal_loss
params1 = {'booster' : 'gbtree',  # 'dart'
          'max_depth': 10,
          # 'objective':'binary:logistic',
          'learning_rate':0.05}
params2 = {'feval':custom_error,     #'focal_loss_eval'         
          'obj':focal_loss}
y_pred, val_result = train_onevsall(x_down_train,y_down_train,x_test,xgbt,params1=params1,params2=params2)
 
print('Write to submission file.')
f = open("submission_xgb{0:.4f}.csv".format(val_result), "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()

*************************************************
Learner No. 1:

[0]	eval-rmse:0.498309	train-rmse:0.496251	eval-1-BMAC:0.329018	train-1-BMAC:0.038402
[1]	eval-rmse:0.497237	train-rmse:0.49253	eval-1-BMAC:0.333873	train-1-BMAC:0.018141
[2]	eval-rmse:0.495618	train-rmse:0.488915	eval-1-BMAC:0.347986	train-1-BMAC:0.019176
[3]	eval-rmse:0.494513	train-rmse:0.485145	eval-1-BMAC:0.338278	train-1-BMAC:0.013458
[4]	eval-rmse:0.493108	train-rmse:0.481422	eval-1-BMAC:0.347537	train-1-BMAC:0.011387
[5]	eval-rmse:0.492113	train-rmse:0.477831	eval-1-BMAC:0.338278	train-1-BMAC:0.007246
[6]	eval-rmse:0.490773	train-rmse:0.474255	eval-1-BMAC:0.314455	train-1-BMAC:0.007246
[7]	eval-rmse:0.489143	train-rmse:0.470968	eval-1-BMAC:0.328119	train-1-BMAC:0.006211
[8]	eval-rmse:0.488475	train-rmse:0.467395	eval-1-BMAC:0.332524	train-1-BMAC:0.005176
[9]	eval-rmse:0.487275	train-rmse:0.463751	eval-1-BMAC:0.337379	train-1-BMAC:0.005176
[10]	eval-rmse:0.486289	train-rmse:0.460263	eval-1-BMAC:0.32767	train-1-BMA

[18]	eval-rmse:0.464647	train-rmse:0.430968	eval-1-BMAC:0.291194	train-1-BMAC:0.006818
[19]	eval-rmse:0.463137	train-rmse:0.42765	eval-1-BMAC:0.291194	train-1-BMAC:0.005682
[20]	eval-rmse:0.461605	train-rmse:0.424377	eval-1-BMAC:0.295824	train-1-BMAC:0.005682
[21]	eval-rmse:0.459701	train-rmse:0.421127	eval-1-BMAC:0.295824	train-1-BMAC:0.003409
[22]	eval-rmse:0.457981	train-rmse:0.417846	eval-1-BMAC:0.301398	train-1-BMAC:0.002273
[23]	eval-rmse:0.456415	train-rmse:0.414639	eval-1-BMAC:0.291194	train-1-BMAC:0.002273
[24]	eval-rmse:0.454914	train-rmse:0.411525	eval-1-BMAC:0.292139	train-1-BMAC:0.003409
[25]	eval-rmse:0.453508	train-rmse:0.408397	eval-1-BMAC:0.301398	train-1-BMAC:0.003409
[26]	eval-rmse:0.452135	train-rmse:0.405408	eval-1-BMAC:0.292139	train-1-BMAC:0.002273
[27]	eval-rmse:0.450672	train-rmse:0.40227	eval-1-BMAC:0.302343	train-1-BMAC:0.002273
[28]	eval-rmse:0.449086	train-rmse:0.399241	eval-1-BMAC:0.306973	train-1-BMAC:0.002273
[29]	eval-rmse:0.447574	train-rmse:0.396254	e

[37]	eval-rmse:0.456188	train-rmse:0.374861	eval-1-BMAC:0.332075	train-1-BMAC:0
[38]	eval-rmse:0.455272	train-rmse:0.372268	eval-1-BMAC:0.322816	train-1-BMAC:0
[39]	eval-rmse:0.454397	train-rmse:0.369708	eval-1-BMAC:0.308702	train-1-BMAC:0
[40]	eval-rmse:0.453897	train-rmse:0.367208	eval-1-BMAC:0.322816	train-1-BMAC:0
[41]	eval-rmse:0.453123	train-rmse:0.364723	eval-1-BMAC:0.308702	train-1-BMAC:0
[42]	eval-rmse:0.452482	train-rmse:0.362278	eval-1-BMAC:0.313556	train-1-BMAC:0
[43]	eval-rmse:0.451897	train-rmse:0.359879	eval-1-BMAC:0.322366	train-1-BMAC:0
[44]	eval-rmse:0.450868	train-rmse:0.357506	eval-1-BMAC:0.322366	train-1-BMAC:0
[45]	eval-rmse:0.450115	train-rmse:0.355162	eval-1-BMAC:0.313107	train-1-BMAC:0
[46]	eval-rmse:0.449247	train-rmse:0.35285	eval-1-BMAC:0.313107	train-1-BMAC:0
[47]	eval-rmse:0.44874	train-rmse:0.350569	eval-1-BMAC:0.313107	train-1-BMAC:0
[48]	eval-rmse:0.448067	train-rmse:0.348292	eval-1-BMAC:0.317961	train-1-BMAC:0
[49]	eval-rmse:0.447423	train-rmse:0.34605

In [48]:
# with default_loss
params1 = {'booster' : 'gbtree',  # 'dart'
          'max_depth': 10,
          # 'objective':'binary:logistic',
          'learning_rate':0.05}
params2 = {'feval':custom_error}     #'focal_loss_eval'         
y_pred, val_result = train_onevsall(x_down_train,y_down_train,x_test,xgbt,params1=params1,params2=params2)
 
print('Write to submission file.')
f = open("submission_xgb{0:.4f}.csv".format(val_result), "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()

*************************************************
Learner No. 1:

[0]	eval-rmse:0.492674	train-rmse:0.477151	eval-1-BMAC:0.376214	train-1-BMAC:0.009416
[1]	eval-rmse:0.486328	train-rmse:0.45545	eval-1-BMAC:0.385023	train-1-BMAC:0.011437
[2]	eval-rmse:0.479894	train-rmse:0.435146	eval-1-BMAC:0.380618	train-1-BMAC:0.008331
[3]	eval-rmse:0.477299	train-rmse:0.41676	eval-1-BMAC:0.366505	train-1-BMAC:0.008331
[4]	eval-rmse:0.472289	train-rmse:0.399195	eval-1-BMAC:0.366505	train-1-BMAC:0.006261
[5]	eval-rmse:0.469314	train-rmse:0.382968	eval-1-BMAC:0.347986	train-1-BMAC:0.006261
[6]	eval-rmse:0.46619	train-rmse:0.368273	eval-1-BMAC:0.352391	train-1-BMAC:0.007296
[7]	eval-rmse:0.464224	train-rmse:0.353234	eval-1-BMAC:0.352391	train-1-BMAC:0.008331
[8]	eval-rmse:0.462152	train-rmse:0.340391	eval-1-BMAC:0.356796	train-1-BMAC:0.007789
[9]	eval-rmse:0.458972	train-rmse:0.327139	eval-1-BMAC:0.333423	train-1-BMAC:0.006753
[10]	eval-rmse:0.456329	train-rmse:0.314023	eval-1-BMAC:0.328569	train-1-BMAC

[17]	eval-rmse:0.379374	train-rmse:0.221709	eval-1-BMAC:0.219766	train-1-BMAC:0.001136
[18]	eval-rmse:0.378202	train-rmse:0.211566	eval-1-BMAC:0.240174	train-1-BMAC:0.001136
[19]	eval-rmse:0.375443	train-rmse:0.201921	eval-1-BMAC:0.240174	train-1-BMAC:0.001136
[20]	eval-rmse:0.37432	train-rmse:0.19265	eval-1-BMAC:0.240174	train-1-BMAC:0.001136
[21]	eval-rmse:0.372288	train-rmse:0.18391	eval-1-BMAC:0.259637	train-1-BMAC:0.001136
[22]	eval-rmse:0.370129	train-rmse:0.175619	eval-1-BMAC:0.255008	train-1-BMAC:0
[23]	eval-rmse:0.367825	train-rmse:0.168026	eval-1-BMAC:0.260582	train-1-BMAC:0
[24]	eval-rmse:0.366648	train-rmse:0.160435	eval-1-BMAC:0.260582	train-1-BMAC:0
[25]	eval-rmse:0.366152	train-rmse:0.153552	eval-1-BMAC:0.260582	train-1-BMAC:0
[26]	eval-rmse:0.364647	train-rmse:0.147295	eval-1-BMAC:0.255952	train-1-BMAC:0
[27]	eval-rmse:0.36412	train-rmse:0.140967	eval-1-BMAC:0.250378	train-1-BMAC:0
[28]	eval-rmse:0.363158	train-rmse:0.135011	eval-1-BMAC:0.255952	train-1-BMAC:0
[29]	eval

[36]	eval-rmse:0.43837	train-rmse:0.093251	eval-1-BMAC:0.359853	train-1-BMAC:0
[37]	eval-rmse:0.438168	train-rmse:0.089313	eval-1-BMAC:0.359853	train-1-BMAC:0
[38]	eval-rmse:0.437777	train-rmse:0.085472	eval-1-BMAC:0.359853	train-1-BMAC:0
[39]	eval-rmse:0.437292	train-rmse:0.081801	eval-1-BMAC:0.359853	train-1-BMAC:0
[40]	eval-rmse:0.437263	train-rmse:0.078306	eval-1-BMAC:0.369112	train-1-BMAC:0
[41]	eval-rmse:0.437142	train-rmse:0.075024	eval-1-BMAC:0.369112	train-1-BMAC:0
[42]	eval-rmse:0.437076	train-rmse:0.071938	eval-1-BMAC:0.369112	train-1-BMAC:0
[43]	eval-rmse:0.436841	train-rmse:0.068956	eval-1-BMAC:0.369112	train-1-BMAC:0
[44]	eval-rmse:0.436717	train-rmse:0.066154	eval-1-BMAC:0.369112	train-1-BMAC:0
[45]	eval-rmse:0.436578	train-rmse:0.063417	eval-1-BMAC:0.369112	train-1-BMAC:0
[46]	eval-rmse:0.435967	train-rmse:0.060792	eval-1-BMAC:0.369112	train-1-BMAC:0
[47]	eval-rmse:0.435282	train-rmse:0.058311	eval-1-BMAC:0.369112	train-1-BMAC:0
[48]	eval-rmse:0.434891	train-rmse:0.0559

In [51]:
def train_xgb_focal(x_train,y_train,x_test,params1,n_iter=10,choose_best=True,params2=None):
    best = 0
    results = []
    for i in range(n_iter):
        y_pred, result = train_onevsall(x_train,y_train,x_test,xgbt,params1=params1,params2=params2)
        results.append(result)
        print("val_BMAC:",result)
        if choose_best & (result <= best):
                continue
        best = result
        f = open("submission_xgb_focal{0:.4f}.csv".format(best), "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,x))
        f.close()
    return results

In [54]:
params1 = {'booster' : 'gbtree',  # 'dart'
          'max_depth': 10,
          'objective':'binary:logistic',
          'learning_rate':0.05}
params2 = {'feval':custom_error} 
feval_val = focal_loss_eval      #'custom_error'  
feval_train = balanced_accuracy_score
obj = focal_loss

train_xgb_focal(x_down_train,y_down_train,x_test,params1=params1,n_iter=10,choose_best=True,params2=params2)
    


*************************************************
Learner No. 1:

[0]	eval-error:0.343949	train-error:0.064057	eval-1-BMAC:0.386372	train-1-BMAC:0.067044
[1]	eval-error:0.356688	train-error:0.026335	eval-1-BMAC:0.408396	train-1-BMAC:0.033866
[2]	eval-error:0.318471	train-error:0.019929	eval-1-BMAC:0.366055	train-1-BMAC:0.025535
[3]	eval-error:0.305732	train-error:0.014235	eval-1-BMAC:0.347537	train-1-BMAC:0.017747
[4]	eval-error:0.280255	train-error:0.007829	eval-1-BMAC:0.323714	train-1-BMAC:0.010401
[5]	eval-error:0.273885	train-error:0.006406	eval-1-BMAC:0.32767	train-1-BMAC:0.008331
[6]	eval-error:0.286624	train-error:0.007829	eval-1-BMAC:0.341784	train-1-BMAC:0.011387
[7]	eval-error:0.286624	train-error:0.004982	eval-1-BMAC:0.332974	train-1-BMAC:0.007246
[8]	eval-error:0.318471	train-error:0.003559	eval-1-BMAC:0.374865	train-1-BMAC:0.005176
[9]	eval-error:0.305732	train-error:0.002847	eval-1-BMAC:0.365156	train-1-BMAC:0.004141
[10]	eval-error:0.299363	train-error:0.001423	eval-1-BM

[20]	eval-error:0.197452	train-error:0.003559	eval-1-BMAC:0.277305	train-1-BMAC:0.005682
[21]	eval-error:0.203822	train-error:0.003559	eval-1-BMAC:0.287509	train-1-BMAC:0.005682
[22]	eval-error:0.197452	train-error:0.003559	eval-1-BMAC:0.28288	train-1-BMAC:0.005682
[23]	eval-error:0.184713	train-error:0.003559	eval-1-BMAC:0.262472	train-1-BMAC:0.005682
[24]	eval-error:0.184713	train-error:0.002847	eval-1-BMAC:0.262472	train-1-BMAC:0.004545
[25]	eval-error:0.178344	train-error:0.002847	eval-1-BMAC:0.252268	train-1-BMAC:0.004545
[26]	eval-error:0.178344	train-error:0.002847	eval-1-BMAC:0.252268	train-1-BMAC:0.004545
[27]	eval-error:0.178344	train-error:0.002847	eval-1-BMAC:0.252268	train-1-BMAC:0.004545
[28]	eval-error:0.178344	train-error:0.002135	eval-1-BMAC:0.252268	train-1-BMAC:0.003409
[29]	eval-error:0.178344	train-error:0.002135	eval-1-BMAC:0.252268	train-1-BMAC:0.003409
[30]	eval-error:0.165605	train-error:0.002135	eval-1-BMAC:0.237434	train-1-BMAC:0.003409
[31]	eval-error:0.1656

[37]	eval-error:0.254777	train-error:0	eval-1-BMAC:0.291082	train-1-BMAC:0
[38]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[39]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[40]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[41]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[42]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[43]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[44]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[45]	eval-error:0.254777	train-error:0	eval-1-BMAC:0.291082	train-1-BMAC:0
[46]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.295937	train-1-BMAC:0
[47]	eval-error:0.254777	train-error:0	eval-1-BMAC:0.291082	train-1-BMAC:0
[48]	eval-error:0.254777	train-error:0	eval-1-BMAC:0.291082	train-1-BMAC:0
[49]	eval-error:0.267516	train-error:0	eval-1-BMAC:0.305196	train-1-BMAC:0
[50]	eval-error:0.261146	

[59]	eval-error:0.280255	train-error:0	eval-1-BMAC:0.358954	train-1-BMAC:0
[60]	eval-error:0.273885	train-error:0	eval-1-BMAC:0.345289	train-1-BMAC:0
[61]	eval-error:0.273885	train-error:0	eval-1-BMAC:0.345289	train-1-BMAC:0
[62]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.331176	train-1-BMAC:0
[63]	eval-error:0.261146	train-error:0	eval-1-BMAC:0.331176	train-1-BMAC:0
[64]	eval-error:0.267516	train-error:0	eval-1-BMAC:0.340435	train-1-BMAC:0
[65]	eval-error:0.273885	train-error:0	eval-1-BMAC:0.349694	train-1-BMAC:0
[66]	eval-error:0.280255	train-error:0	eval-1-BMAC:0.358954	train-1-BMAC:0
[67]	eval-error:0.273885	train-error:0	eval-1-BMAC:0.349694	train-1-BMAC:0
[68]	eval-error:0.273885	train-error:0	eval-1-BMAC:0.349694	train-1-BMAC:0
[69]	eval-error:0.280255	train-error:0	eval-1-BMAC:0.358954	train-1-BMAC:0
[70]	eval-error:0.280255	train-error:0	eval-1-BMAC:0.358954	train-1-BMAC:0
[71]	eval-error:0.273885	train-error:0	eval-1-BMAC:0.354099	train-1-BMAC:0
[72]	eval-error:0.273885	

[77]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.260582	train-1-BMAC:0
[78]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.260582	train-1-BMAC:0
[79]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.260582	train-1-BMAC:0
----------binary classifier------------
balanced_accuracy_score on validation data:0.739418
*************************************************
Learner No. 3:

[0]	eval-error:0.280255	train-error:0.045552	eval-1-BMAC:0.3105	train-1-BMAC:0.050034
[1]	eval-error:0.286624	train-error:0.030605	eval-1-BMAC:0.332974	train-1-BMAC:0.037667
[2]	eval-error:0.292994	train-error:0.022064	eval-1-BMAC:0.346638	train-1-BMAC:0.026706
[3]	eval-error:0.280255	train-error:0.01637	eval-1-BMAC:0.332524	train-1-BMAC:0.019894
[4]	eval-error:0.273885	train-error:0.017794	eval-1-BMAC:0.31886	train-1-BMAC:0.022464
[5]	eval-error:0.267516	train-error:0.014235	eval-1-BMAC:0.318411	train-1-BMAC:0.01926
[6]	eval-error:0.261146	train-error:0.014947	eval-1-BMAC:0.309151	train-1-BMAC:0.020297
[7]	ev

...

[13]	eval-error:0.22293	train-error:0.002847	eval-1-BMAC:0.271215	train-1-BMAC:0.004149
[14]	eval-error:0.235669	train-error:0.002847	eval-1-BMAC:0.289734	train-1-BMAC:0.004149
[15]	eval-error:0.229299	train-error:0.002135	eval-1-BMAC:0.27607	train-1-BMAC:0.003112
[16]	eval-error:0.235669	train-error:0.002135	eval-1-BMAC:0.285329	train-1-BMAC:0.003112
[17]	eval-error:0.210191	train-error:0.002135	eval-1-BMAC:0.257102	train-1-BMAC:0.003112
[18]	eval-error:0.22293	train-error:0.001423	eval-1-BMAC:0.27562	train-1-BMAC:0.002075
[19]	eval-error:0.216561	train-error:0.001423	eval-1-BMAC:0.266361	train-1-BMAC:0.002075
[20]	eval-error:0.210191	train-error:0.001423	eval-1-BMAC:0.252697	train-1-BMAC:0.002075
[21]	eval-error:0.210191	train-error:0.001423	eval-1-BMAC:0.252697	train-1-BMAC:0.002075
[22]	eval-error:0.203822	train-error:0.001423	eval-1-BMAC:0.239033	train-1-BMAC:0.002075
[23]	eval-error:0.210191	train-error:0.000712	eval-1-BMAC:0.252697	train-1-BMAC:0.001037
[24]	eval-error:0.210191	

[32]	eval-error:0.33121	train-error:0	eval-1-BMAC:0.393384	train-1-BMAC:0
[33]	eval-error:0.33758	train-error:0	eval-1-BMAC:0.398238	train-1-BMAC:0
[34]	eval-error:0.33758	train-error:0	eval-1-BMAC:0.402643	train-1-BMAC:0
[35]	eval-error:0.343949	train-error:0	eval-1-BMAC:0.411902	train-1-BMAC:0
[36]	eval-error:0.343949	train-error:0	eval-1-BMAC:0.411902	train-1-BMAC:0
[37]	eval-error:0.343949	train-error:0	eval-1-BMAC:0.411902	train-1-BMAC:0
[38]	eval-error:0.350318	train-error:0	eval-1-BMAC:0.421161	train-1-BMAC:0
[39]	eval-error:0.350318	train-error:0	eval-1-BMAC:0.421161	train-1-BMAC:0
[40]	eval-error:0.343949	train-error:0	eval-1-BMAC:0.416307	train-1-BMAC:0
[41]	eval-error:0.350318	train-error:0	eval-1-BMAC:0.421161	train-1-BMAC:0
[42]	eval-error:0.324841	train-error:0	eval-1-BMAC:0.388529	train-1-BMAC:0
[43]	eval-error:0.33121	train-error:0	eval-1-BMAC:0.397789	train-1-BMAC:0
[44]	eval-error:0.33758	train-error:0	eval-1-BMAC:0.407048	train-1-BMAC:0
[45]	eval-error:0.343949	train

[53]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[54]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[55]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[56]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[57]	eval-error:0.191083	train-error:0	eval-1-BMAC:0.22808	train-1-BMAC:0
[58]	eval-error:0.191083	train-error:0	eval-1-BMAC:0.22808	train-1-BMAC:0
[59]	eval-error:0.191083	train-error:0	eval-1-BMAC:0.22808	train-1-BMAC:0
[60]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[61]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[62]	eval-error:0.197452	train-error:0	eval-1-BMAC:0.238284	train-1-BMAC:0
[63]	eval-error:0.191083	train-error:0	eval-1-BMAC:0.233655	train-1-BMAC:0
[64]	eval-error:0.184713	train-error:0	eval-1-BMAC:0.22345	train-1-BMAC:0
[65]	eval-error:0.191083	train-error:0	eval-1-BMAC:0.233655	train-1-BMAC:0
[66]	eval-error:0.191083	trai

[76]	eval-error:0.312102	train-error:0	eval-1-BMAC:0.356796	train-1-BMAC:0
[77]	eval-error:0.312102	train-error:0	eval-1-BMAC:0.356796	train-1-BMAC:0
[78]	eval-error:0.312102	train-error:0	eval-1-BMAC:0.356796	train-1-BMAC:0
[79]	eval-error:0.312102	train-error:0	eval-1-BMAC:0.356796	train-1-BMAC:0
----------binary classifier------------
balanced_accuracy_score on validation data:0.643204
----------ove vs all classifier------------
balanced_accuracy_score on validation data:0.646296
val_BMAC: 0.6462962962962963


[0.645679012345679,
 0.6432098765432098,
 0.6703703703703704,
 0.7129629629629629,
 0.6734567901234567,
 0.6882716049382717,
 0.6111111111111112,
 0.7290123456790122,
 0.6098765432098766,
 0.6462962962962963]

In [58]:
# xgbt(x_down_train,y_down_train,x_test,xgbt,params)

In [62]:
# y_pred = train_onevsall(x_down_train,y_down_train,x_test,xgbt,params)

In [ ]:
# y_train = 1*(y_train!=0)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train,test_size=0.2)
# dtrain = xgb.DMatrix(x_train, label=y_train)
# dtest = xgb.DMatrix(x_val, label=y_val)
# watchlist = [(dtest, 'eval'), (dtrain, 'train')]
# num_round = 50
# bst = xgb.train(params, dtrain, num_round, watchlist)
#                 # , obj=focal_loss,
#                 # feval=focal_loss_eval)
# y_val_p = bst.predict(dtest)
# balanced_accuracy_score(y_val, [1 if x>0.5 else 0 for x in y_val_p])
# # len(bst.predict(dtest))

In [65]:
# try PCA
n_components = [500,400,300,200,100,80,50,10]
# n_components= [500]
results_PCA = {}
for k in n_components:
    pca = PCA(n_components=k)
    pca_x_train = pca.fit_transform(x_down_train)
    pca_x_test = pca.transform(x_test)
    print("The first %d principle components explain %f overall variance." % (k,sum(pca.explained_variance_ratio_)))
    results = train_xgb(pca_x_train,y_down_train,pca_x_test,params,n_iter=10,choose_best=True)
    results_PCA[k] = np.mean(results)
print("*********************************")
print("Performance on validation set for different PCA n_components:")
print(results_PCA)

The first 500 principle components explain 0.947898 overall variance.
[0]	train-merror:0.350865	valid-merror:0.537356	train-1-BMAC:0.350292	valid-1-BMAC:0.535959
Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.278098	valid-merror:0.477012	train-1-BMAC:0.277081	valid-1-BMAC:0.476014
[2]	train-merror:0.221182	valid-merror:0.413793	train-1-BMAC:0.221113	valid-1-BMAC:0.413805
[3]	train-merror:0.194524	valid-merror:0.382184	train-1-BMAC:0.194357	valid-1-BMAC:0.383086
[4]	train-merror:0.164265	valid-merror:0.382184	train-1-BMAC:0.16466	valid-1-BMAC:0.38304
[5]	train-merror:0.158501	valid-merror:0.390805	train-1-BMAC:0.158724	valid-1-BMAC:0.391443
[6]	train-merror:0.157781	valid-merror:0.367816	train-1-BMAC:0.157817	valid-1-BMAC:0.368871
[7]	train-merror:0.146254	valid-merror:0.376437	train-1-BMAC:0.146082	valid-1-BMAC:0.377578
[8]	train-merror:0.149135	valid-merror:0.373563

val_BMAC: 0.6684336559790282
[0]	train-merror:0.320946	valid-merror:0.468468	train-1-BMAC:0.320604	valid-1-BMAC:0.464758
Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.266892	valid-merror:0.518018	train-1-BMAC:0.267826	valid-1-BMAC:0.514726
[2]	train-merror:0.231982	valid-merror:0.454955	train-1-BMAC:0.232779	valid-1-BMAC:0.451714
[3]	train-merror:0.177928	valid-merror:0.414414	train-1-BMAC:0.178303	valid-1-BMAC:0.41119
[4]	train-merror:0.163288	valid-merror:0.445946	train-1-BMAC:0.163353	valid-1-BMAC:0.444174
[5]	train-merror:0.141892	valid-merror:0.40991	train-1-BMAC:0.142269	valid-1-BMAC:0.40742
[6]	train-merror:0.135135	valid-merror:0.387387	train-1-BMAC:0.135075	valid-1-BMAC:0.386106
[7]	train-merror:0.120495	valid-merror:0.387387	train-1-BMAC:0.120867	valid-1-BMAC:0.385216
[8]	train-merror:0.120495	valid-merror:0.364865	train-1-BMAC:0.120761	valid-1-BMAC:0.3618

[2]	train-merror:0.211268	valid-merror:0.401408	train-1-BMAC:0.210189	valid-1-BMAC:0.400124
[3]	train-merror:0.183099	valid-merror:0.394366	train-1-BMAC:0.182512	valid-1-BMAC:0.394094
[4]	train-merror:0.170775	valid-merror:0.359155	train-1-BMAC:0.171363	valid-1-BMAC:0.358534
[5]	train-merror:0.133803	valid-merror:0.352113	train-1-BMAC:0.134649	valid-1-BMAC:0.350186
[6]	train-merror:0.121479	valid-merror:0.338028	train-1-BMAC:0.122193	valid-1-BMAC:0.33658
[7]	train-merror:0.121479	valid-merror:0.34507	train-1-BMAC:0.12203	valid-1-BMAC:0.344929
[8]	train-merror:0.121479	valid-merror:0.338028	train-1-BMAC:0.122357	valid-1-BMAC:0.338126
[9]	train-merror:0.114437	valid-merror:0.316901	train-1-BMAC:0.115846	valid-1-BMAC:0.316945
[10]	train-merror:0.117958	valid-merror:0.309859	train-1-BMAC:0.118775	valid-1-BMAC:0.310142
[11]	train-merror:0.107394	valid-merror:0.309859	train-1-BMAC:0.108518	valid-1-BMAC:0.309369
[12]	train-merror:0.100352	valid-merror:0.316901	train-1-BMAC:0.101191	valid-1-BM

[11]	train-merror:0.093664	valid-merror:0.395604	train-1-BMAC:0.093782	valid-1-BMAC:0.397479
[12]	train-merror:0.090909	valid-merror:0.373626	train-1-BMAC:0.091398	valid-1-BMAC:0.37449
[13]	train-merror:0.082645	valid-merror:0.395604	train-1-BMAC:0.083115	valid-1-BMAC:0.395996
[14]	train-merror:0.068871	valid-merror:0.417582	train-1-BMAC:0.069215	valid-1-BMAC:0.418242
[15]	train-merror:0.068871	valid-merror:0.406593	train-1-BMAC:0.069499	valid-1-BMAC:0.40749
[16]	train-merror:0.066116	valid-merror:0.373626	train-1-BMAC:0.066265	valid-1-BMAC:0.373749
[17]	train-merror:0.060606	valid-merror:0.384615	train-1-BMAC:0.061215	valid-1-BMAC:0.385243
[18]	train-merror:0.063361	valid-merror:0.395604	train-1-BMAC:0.063599	valid-1-BMAC:0.395996
[19]	train-merror:0.063361	valid-merror:0.384615	train-1-BMAC:0.063882	valid-1-BMAC:0.384501
[20]	train-merror:0.066116	valid-merror:0.395604	train-1-BMAC:0.066549	valid-1-BMAC:0.396737
[21]	train-merror:0.060606	valid-merror:0.406593	train-1-BMAC:0.061215	v

[9]	train-merror:0.102703	valid-merror:0.531915	train-1-BMAC:0.104715	valid-1-BMAC:0.533333
[10]	train-merror:0.108108	valid-merror:0.531915	train-1-BMAC:0.111202	valid-1-BMAC:0.533333
[11]	train-merror:0.108108	valid-merror:0.531915	train-1-BMAC:0.110563	valid-1-BMAC:0.534722
[12]	train-merror:0.07027	valid-merror:0.510638	train-1-BMAC:0.072186	valid-1-BMAC:0.5125
[13]	train-merror:0.075676	valid-merror:0.489362	train-1-BMAC:0.077394	valid-1-BMAC:0.488889
[14]	train-merror:0.108108	valid-merror:0.489362	train-1-BMAC:0.111842	valid-1-BMAC:0.490278
[15]	train-merror:0.091892	valid-merror:0.446809	train-1-BMAC:0.094298	valid-1-BMAC:0.448611
[16]	train-merror:0.07027	valid-merror:0.468085	train-1-BMAC:0.073465	valid-1-BMAC:0.469444
[17]	train-merror:0.075676	valid-merror:0.489362	train-1-BMAC:0.078673	valid-1-BMAC:0.490278
[18]	train-merror:0.07027	valid-merror:0.446809	train-1-BMAC:0.072186	valid-1-BMAC:0.447222
[19]	train-merror:0.07027	valid-merror:0.489362	train-1-BMAC:0.072825	valid-

...

[10]	train-merror:0.176724	valid-merror:0.275862	train-1-BMAC:0.176852	valid-1-BMAC:0.272222
[11]	train-merror:0.172414	valid-merror:0.327586	train-1-BMAC:0.172685	valid-1-BMAC:0.322222
[12]	train-merror:0.185345	valid-merror:0.293103	train-1-BMAC:0.186111	valid-1-BMAC:0.288889
[13]	train-merror:0.172414	valid-merror:0.293103	train-1-BMAC:0.173611	valid-1-BMAC:0.288889
[14]	train-merror:0.176724	valid-merror:0.310345	train-1-BMAC:0.178241	valid-1-BMAC:0.305556
[15]	train-merror:0.176724	valid-merror:0.344828	train-1-BMAC:0.177778	valid-1-BMAC:0.340741
[16]	train-merror:0.159483	valid-merror:0.327586	train-1-BMAC:0.160648	valid-1-BMAC:0.324074
[17]	train-merror:0.150862	valid-merror:0.327586	train-1-BMAC:0.152778	valid-1-BMAC:0.324074
[18]	train-merror:0.150862	valid-merror:0.275862	train-1-BMAC:0.152315	valid-1-BMAC:0.272222
[19]	train-merror:0.159483	valid-merror:0.293103	train-1-BMAC:0.161574	valid-1-BMAC:0.288889
Stopping. Best iteration:
[4]	train-merror:0.211207	valid-merror:0.275

Multiple eval metrics have been passed: 'valid-1-BMAC' will be used for early stopping.

Will train until valid-1-BMAC hasn't improved in 15 rounds.
[1]	train-merror:0.298198	valid-merror:0.420863	train-1-BMAC:0.299123	valid-1-BMAC:0.421142
[2]	train-merror:0.3	valid-merror:0.417266	train-1-BMAC:0.300968	valid-1-BMAC:0.417098
[3]	train-merror:0.282883	valid-merror:0.406475	train-1-BMAC:0.283048	valid-1-BMAC:0.406535
[4]	train-merror:0.272072	valid-merror:0.410072	train-1-BMAC:0.271963	valid-1-BMAC:0.409508
[5]	train-merror:0.269369	valid-merror:0.406475	train-1-BMAC:0.269347	valid-1-BMAC:0.406249
[6]	train-merror:0.267568	valid-merror:0.420863	train-1-BMAC:0.267868	valid-1-BMAC:0.42057
[7]	train-merror:0.263063	valid-merror:0.410072	train-1-BMAC:0.263232	valid-1-BMAC:0.410799
[8]	train-merror:0.266667	valid-merror:0.413669	train-1-BMAC:0.266472	valid-1-BMAC:0.413875
[9]	train-merror:0.251351	valid-merror:0.428058	train-1-BMAC:0.251189	valid-1-BMAC:0.428519
[10]	train-merror:0.244144	va

[33]	train-merror:0.205634	valid-merror:0.331461	train-1-BMAC:0.206192	valid-1-BMAC:0.328259
[34]	train-merror:0.207042	valid-merror:0.348315	train-1-BMAC:0.2077	valid-1-BMAC:0.345141
[35]	train-merror:0.208451	valid-merror:0.348315	train-1-BMAC:0.209055	valid-1-BMAC:0.345141
[36]	train-merror:0.208451	valid-merror:0.348315	train-1-BMAC:0.209049	valid-1-BMAC:0.345141
[37]	train-merror:0.205634	valid-merror:0.325843	train-1-BMAC:0.20647	valid-1-BMAC:0.323283
[38]	train-merror:0.209859	valid-merror:0.320225	train-1-BMAC:0.210705	valid-1-BMAC:0.317818
[39]	train-merror:0.209859	valid-merror:0.320225	train-1-BMAC:0.210558	valid-1-BMAC:0.317818
[40]	train-merror:0.209859	valid-merror:0.325843	train-1-BMAC:0.210705	valid-1-BMAC:0.32377
[41]	train-merror:0.209859	valid-merror:0.337079	train-1-BMAC:0.210558	valid-1-BMAC:0.334699
[42]	train-merror:0.209859	valid-merror:0.337079	train-1-BMAC:0.210552	valid-1-BMAC:0.335187
[43]	train-merror:0.209859	valid-merror:0.337079	train-1-BMAC:0.210705	val

[21]	train-merror:0.220386	valid-merror:0.351648	train-1-BMAC:0.220696	valid-1-BMAC:0.35095
[22]	train-merror:0.22314	valid-merror:0.318681	train-1-BMAC:0.223363	valid-1-BMAC:0.318692
[23]	train-merror:0.225895	valid-merror:0.32967	train-1-BMAC:0.226029	valid-1-BMAC:0.329445
[24]	train-merror:0.225895	valid-merror:0.340659	train-1-BMAC:0.226313	valid-1-BMAC:0.340198
[25]	train-merror:0.217631	valid-merror:0.318681	train-1-BMAC:0.218029	valid-1-BMAC:0.318692
[26]	train-merror:0.217631	valid-merror:0.318681	train-1-BMAC:0.218029	valid-1-BMAC:0.318692
[27]	train-merror:0.214876	valid-merror:0.318681	train-1-BMAC:0.215363	valid-1-BMAC:0.318692
[28]	train-merror:0.217631	valid-merror:0.32967	train-1-BMAC:0.218029	valid-1-BMAC:0.330597
[29]	train-merror:0.220386	valid-merror:0.318681	train-1-BMAC:0.220696	valid-1-BMAC:0.318692
[30]	train-merror:0.214876	valid-merror:0.318681	train-1-BMAC:0.215363	valid-1-BMAC:0.318692
[31]	train-merror:0.220386	valid-merror:0.32967	train-1-BMAC:0.220696	vali

[20]	train-merror:0.248649	valid-merror:0.340426	train-1-BMAC:0.250457	valid-1-BMAC:0.340278
Stopping. Best iteration:
[5]	train-merror:0.281081	valid-merror:0.319149	train-1-BMAC:0.283626	valid-1-BMAC:0.319444

val_BMAC: 0.6597222222222222
*********************************
Performance on validation set for different PCA n_components:
{500: 0.6116684394826122, 400: 0.6201805401079103, 300: 0.620189505166625, 200: 0.6176846138917281, 100: 0.6298375654100014, 80: 0.6300769476747644, 50: 0.6306631530305908, 10: 0.6354422247644653}


In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'multiclassova',
    'is_unbalance': True,
    #'metric': {'l1', 'l2'},
    'num_class': 3,
    'num_leaves': 10,
    'num_iterations':500,
    'learning_rate': 0.01,
    'feature_fraction': 0.2,
    'bagging_fraction': 0.8,
    'verbose': 0,
    'subsample':0.5,
    'subsample_freq':2
}
# train_lightgbm(x_down_train,y_down_train,params,20,True)

In [ ]:
## for binary classification problem: https://github.com/jhwjhw0123/Imbalance-XGBoost

# from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
# from sklearn.model_selection import GridSearchCV
# 
# xgboster_focal = imb_xgb(special_objective='focal')
# xgboster_weight = imb_xgb(special_objective='weighted')
# CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
# CV_weight_booster = GridSearchCV(xgboster_weight, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
# CV_focal_booster.fit(x_train, y_train)
# CV_weight_booster.fit(x_train, y_train)
# opt_focal_booster = CV_focal_booster.best_estimator_
# opt_weight_booster = CV_weight_booster.best_estimator_
# raw_output = opt_focal_booster.predict(x_test, y=None) 
# sigmoid_output = opt_focal_booster.predict_sigmoid(x_test, y=None) 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#grid of parameters
gridParams = {
    'learning_rate': [0.01,0.02,0.03,0.04,0.05],
    'num_leaves': [5,10,20,30,40,50],
    'boosting_type' : ['dart'],
    'objective' : ['multiclassova'],
    'max_depth' : [5,6,7,8],
    'random_state' : [501], 
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.2,0.5,0.6,0.7],
    'min_split_gain' : [0.01],
    'min_data_in_leaf':[10],
        'task': ['train'],
    'is_unbalance': [True],
    #'metric': {'l1', 'l2'},
    'num_class': [3],
    'num_iterations':[300,500,800],
    'feature_fraction': [0.2,0.5,0.8],
    'bagging_fraction': [0.8],
    'verbose': [0],
    'subsample_freq':[2,4,6,8]
    }
x_train, x_val, y_train, y_val = train_test_split(x_down_train, y_down_train, stratify=y_down_train,test_size = 0.2) # , random_state = 19960503)
# create dataset for lightgbm
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_val, y_val, reference=lgb_train)

print('Start training...')
# train
gbm = lgb.LGBMClassifier()

grid = RandomizedSearchCV(gbm,gridParams)
grid.fit(x_down_train,y_down_train)
y_val_p = grid.predict(x_val)
result = balanced_accuracy_score(y_val, y_val_p)

In [ ]:
balanced_accuracy_score(y_val, y_val_p)

In [ ]:
grid.best_params_

In [ ]:
grid.best_params_

In [ ]:
# # tree-based method
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier 